In [1]:
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
import cv2
import time
from my_functions import *

def generate_notice():
    vehicle_number = vehicle_number_entry.get()
    if vehicle_number.strip() == "":
        messagebox.showerror("Error", "Please enter the vehicle number.")
    else:
        notice = f"Notice: You were detected riding without a helmet on {datetime.now()}. Please ensure you wear a helmet for your safety and to comply with traffic regulations. Violation recorded with vehicle number: {vehicle_number}."
        messagebox.showinfo("Notice Generated", notice)

# Create the main window
root = tk.Tk()
root.title("Helmet Violation Notice Generator")

# Create and place widgets
label = tk.Label(root, text="Enter Vehicle Number:")
label.pack()

vehicle_number_entry = tk.Entry(root)
vehicle_number_entry.pack()

generate_button = tk.Button(root, text="Generate Notice", command=generate_notice)
generate_button.pack()

# Run the application
root.mainloop()

# Rest of your main program goes here...

source = 'video.mp4' 
save_video = True
show_video = True
save_img = False
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, frame_size)

cap = cv2.VideoCapture(source)
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame = cv2.resize(frame, frame_size)
        orifinal_frame = frame.copy()
        frame, results = object_detection(frame) 

        rider_list = []
        head_list = []
        number_list = []

        for result in results:
            x1,y1,x2,y2,cnf, clas = result
            if clas == 0:
                rider_list.append(result)
            elif clas == 1:
                head_list.append(result)
            elif clas == 2:
                number_list.append(result)

        for rdr in rider_list:
            time_stamp = str(time.time())
            x1r, y1r, x2r, y2r, cnfr, clasr = rdr
            for hd in head_list:
                x1h, y1h, x2h, y2h, cnfh, clash = hd
                if inside_box([x1r,y1r,x2r,y2r], [x1h,y1h,x2h,y2h]): 
                    try:
                        head_img = orifinal_frame[y1h:y2h, x1h:x2h]
                        helmet_present = img_classify(head_img)
                    except:
                        helmet_present[0] = None

                    if  helmet_present[0] == True: 
                        frame = cv2.rectangle(frame, (x1h, y1h), (x2h, y2h), (0,255,0), 1)
                        frame = cv2.putText(frame, f'{round(helmet_present[1],1)}', (x1h, y1h+40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    elif helmet_present[0] == None: 
                        frame = cv2.rectangle(frame, (x1h, y1h), (x2h, y2h), (0, 255, 255), 1)
                        frame = cv2.putText(frame, f'{round(helmet_present[1],1)}', (x1h, y1h), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    elif helmet_present[0] == False: 
                        frame = cv2.rectangle(frame, (x1h, y1h), (x2h, y2h), (0, 0, 255), 1)
                        frame = cv2.putText(frame, f'{round(helmet_present[1],1)}', (x1h, y1h+40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                        try:
                            cv2.imwrite(f'riders_pictures/{time_stamp}.jpg', frame[y1r:y2r, x1r:x2r])
                        except:
                            print('could not save rider')

                        for num in number_list:
                            x1_num, y1_num, x2_num, y2_num, conf_num, clas_num = num
                            if inside_box([x1r,y1r,x2r,y2r], [x1_num, y1_num, x2_num, y2_num]):
                                try:
                                    num_img = orifinal_frame[y1_num:y2_num, x1_num:x2_num]
                                    cv2.imwrite(f'number_plates/{time_stamp}_{conf_num}.jpg', num_img)
                                except:
                                    print('could not save number plate')
                                    
        if save_video:
            out.write(frame)
        if save_img:
            cv2.imwrite('saved_frame.jpg', frame)
        if show_video:
            cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    else:
        break

cap.release()
#cv2.destroyAllWindows()
print('Execution completed')


C:\Users\i\anaconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Detected: rider conf: 0.52  bbox: x1:528    y1:110    x2:717    y2:478
Detected: rider conf: 0.63  bbox: x1:532    y1:7    x2:712    y2:483
Detected: head conf: 0.48  bbox: x1:550    y1:7    x2:669    y2:144
Detected: head conf: 0.6  bbox: x1:551    y1:312    x2:658    y2:417
Detected: rider conf: 0.51  bbox: x1:500    y1:303    x2:732    y2:475
Detected: rider conf: 0.64  bbox: x1:498    y1:187    x2:733    y2:481
Detected: head conf: 0.55  bbox: x1:559    y1:201    x2:650    y2:303
Detected: rider conf: 0.5  bbox: x1:513    y1:63    x2:703    y2:492
Detected: number conf: 0.48  bbox: x1:555    y1:424    x2:674    y2:465
Detected: rider conf: 0.4  bbox: x1:523    y1:-4    x2:688    y2:482
Detected: rider conf: 0.59  bbox: x1:336    y1:331    x2:567    y2:479
Detected: head conf: 0.57  bbox: x1:384    y1:334    x2:470    y2:433
Detected: head conf: 0.66  bbox: x1:363    y1:303    x2:456    y2:395
Detected: rider conf: 0.64  bbox: x1:303    y1:288    x2:567    y2:475
Detected: rider con

In [1]:
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
import cv2
import time
from my_functions import *

def generate_notice(vehicle_number):
    if vehicle_number.strip() == "":
        messagebox.showerror("Error", "Please enter the vehicle number.")
    else:
        notice = f"Notice: You were detected riding without a helmet and number plate on {datetime.now()}. Please ensure you wear a helmet for your safety and to comply with traffic regulations. Violation recorded with vehicle number: {vehicle_number}."
        messagebox.showinfo("Notice Generated", notice)

def process_detection(frame, results):
    rider_list = []
    head_list = []
    number_list = []

    for result in results:
        x1,y1,x2,y2,cnf, clas = result
        if clas == 0:
            rider_list.append(result)
        elif clas == 1:
            head_list.append(result)
        elif clas == 2:
            number_list.append(result)

    for rdr in rider_list:
        x1r, y1r, x2r, y2r, cnfr, clasr = rdr
        for hd in head_list:
            x1h, y1h, x2h, y2h, cnfh, clash = hd
            if inside_box([x1r,y1r,x2r,y2r], [x1h,y1h,x2h,y2h]): 
                try:
                    head_img = frame[y1h:y2h, x1h:x2h]
                    helmet_present = img_classify(head_img)
                except:
                    helmet_present[0] = None

                if helmet_present[0] == False:
                    for num in number_list:
                        x1_num, y1_num, x2_num, y2_num, conf_num, clas_num = num
                        if inside_box([x1r,y1r,x2r,y2r], [x1_num, y1_num, x2_num, y2_num]):
                            vehicle_number = extract_number_plate(frame, num)
                            generate_notice(vehicle_number)
                            return
    # If no violation detected, return None
    return None

def main():
    source = 'video.mp4' 
    save_video = True
    show_video = True
    save_img = False
    frame_size = (640, 480)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi', fourcc, 20.0, frame_size)

    cap = cv2.VideoCapture(source)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frame = cv2.resize(frame, frame_size)
            original_frame = frame.copy()
            frame, results = object_detection(frame)

            if process_detection(original_frame, results):
                # If violation detected, process_detection will generate notice
                pass

            if save_video:
                out.write(frame)
            if save_img:
                cv2.imwrite('saved_frame.jpg', frame)
            if show_video:
                cv2.imshow('Frame', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        else:
            break

    cap.release()
    #cv2.destroyAllWindows()
    print('Execution completed')

if __name__ == "__main__":
    main()


C:\Users\i\anaconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Detected: head conf: 0.43  bbox: x1:436    y1:15    x2:536    y2:140
Detected: rider conf: 0.37  bbox: x1:414    y1:3    x2:576    y2:477
Detected: head conf: 0.62  bbox: x1:442    y1:311    x2:529    y2:414
Detected: rider conf: 0.56  bbox: x1:393    y1:301    x2:582    y2:477
Detected: head conf: 0.38  bbox: x1:397    y1:285    x2:574    y2:467
Detected: head conf: 0.65  bbox: x1:445    y1:200    x2:525    y2:306
Detected: rider conf: 0.59  bbox: x1:403    y1:191    x2:582    y2:478
Detected: rider conf: 0.66  bbox: x1:257    y1:334    x2:459    y2:480
Detected: head conf: 0.43  bbox: x1:307    y1:337    x2:385    y2:428
Detected: rider conf: 0.54  bbox: x1:235    y1:286    x2:460    y2:473
Detected: head conf: 0.4  bbox: x1:281    y1:303    x2:378    y2:389
Detected: rider conf: 0.56  bbox: x1:228    y1:261    x2:432    y2:479
Detected: rider conf: 0.46  bbox: x1:228    y1:233    x2:419    y2:469
Detected: rider conf: 0.37  bbox: x1:223    y1:183    x2:399    y2:475
Detected: rider 

Detected: rider conf: 0.36  bbox: x1:124    y1:263    x2:310    y2:483
Detected: rider conf: 0.63  bbox: x1:-6    y1:331    x2:172    y2:480
Detected: rider conf: 0.54  bbox: x1:5    y1:300    x2:184    y2:484
Detected: rider conf: 0.42  bbox: x1:14    y1:245    x2:204    y2:495
Detected: rider conf: 0.59  bbox: x1:39    y1:229    x2:214    y2:488
Detected: rider conf: 0.66  bbox: x1:51    y1:186    x2:225    y2:482
Detected: rider conf: 0.61  bbox: x1:65    y1:156    x2:235    y2:482
Detected: rider conf: 0.5  bbox: x1:139    y1:328    x2:338    y2:469
Detected: rider conf: 0.42  bbox: x1:-5    y1:210    x2:189    y2:472
Detected: rider conf: 0.68  bbox: x1:176    y1:310    x2:372    y2:476
Detected: rider conf: 0.68  bbox: x1:183    y1:261    x2:373    y2:476
Detected: rider conf: 0.55  bbox: x1:193    y1:216    x2:369    y2:475
Detected: rider conf: 0.46  bbox: x1:49    y1:-4    x2:171    y2:287
Detected: rider conf: 0.51  bbox: x1:76    y1:366    x2:270    y2:476
Detected: rider co

Detected: rider conf: 0.4  bbox: x1:-1    y1:346    x2:217    y2:478
Detected: rider conf: 0.4  bbox: x1:-9    y1:237    x2:196    y2:491
Detected: rider conf: 0.48  bbox: x1:-1    y1:-4    x2:87    y2:205
Detected: rider conf: 0.41  bbox: x1:118    y1:337    x2:310    y2:476
Detected: rider conf: 0.67  bbox: x1:307    y1:176    x2:462    y2:484
Detected: head conf: 0.5  bbox: x1:356    y1:180    x2:419    y2:274
Detected: head conf: 0.51  bbox: x1:84    y1:438    x2:152    y2:482
Detected: rider conf: 0.4  bbox: x1:69    y1:349    x2:200    y2:477
Detected: head conf: 0.37  bbox: x1:74    y1:339    x2:193    y2:481
Detected: rider conf: 0.58  bbox: x1:65    y1:304    x2:225    y2:482
Detected: rider conf: 0.54  bbox: x1:81    y1:257    x2:224    y2:487
Detected: rider conf: 0.4  bbox: x1:88    y1:204    x2:224    y2:485
Detected: rider conf: 0.57  bbox: x1:564    y1:44    x2:638    y2:491
Detected: rider conf: 0.56  bbox: x1:552    y1:-1    x2:637    y2:481
Detected: rider conf: 0.42 

Detected: rider conf: 0.52  bbox: x1:182    y1:358    x2:354    y2:478
Detected: rider conf: 0.42  bbox: x1:163    y1:319    x2:337    y2:474
Detected: rider conf: 0.37  bbox: x1:158    y1:274    x2:327    y2:487
Detected: rider conf: 0.4  bbox: x1:154    y1:256    x2:325    y2:485
Detected: rider conf: 0.38  bbox: x1:158    y1:211    x2:303    y2:484
Detected: rider conf: 0.51  bbox: x1:352    y1:1    x2:471    y2:109
Detected: head conf: 0.64  bbox: x1:58    y1:352    x2:185    y2:475
Detected: rider conf: 0.62  bbox: x1:47    y1:358    x2:198    y2:477
Detected: rider conf: 0.47  bbox: x1:33    y1:307    x2:191    y2:479
Detected: rider conf: 0.56  bbox: x1:31    y1:259    x2:187    y2:480
Detected: head conf: 0.44  bbox: x1:62    y1:262    x2:144    y2:346
Detected: rider conf: 0.47  bbox: x1:21    y1:215    x2:178    y2:478
Detected: rider conf: 0.55  bbox: x1:442    y1:343    x2:630    y2:476
Detected: head conf: 0.53  bbox: x1:461    y1:332    x2:611    y2:478
Detected: rider co

Detected: rider conf: 0.57  bbox: x1:512    y1:31    x2:632    y2:428
Detected: rider conf: 0.38  bbox: x1:518    y1:12    x2:640    y2:470
Detected: rider conf: 0.54  bbox: x1:-2    y1:225    x2:115    y2:477
Detected: rider conf: 0.43  bbox: x1:284    y1:0    x2:388    y2:106
Detected: rider conf: 0.35  bbox: x1:93    y1:-3    x2:200    y2:358
Detected: head conf: 0.51  bbox: x1:213    y1:405    x2:288    y2:482
Detected: rider conf: 0.48  bbox: x1:161    y1:361    x2:325    y2:476
Detected: rider conf: 0.52  bbox: x1:134    y1:311    x2:314    y2:482
Detected: rider conf: 0.45  bbox: x1:117    y1:266    x2:299    y2:487
Detected: rider conf: 0.48  bbox: x1:106    y1:235    x2:283    y2:482
Detected: rider conf: 0.43  bbox: x1:97    y1:192    x2:265    y2:479
Detected: rider conf: 0.67  bbox: x1:281    y1:0    x2:377    y2:86
Detected: rider conf: 0.69  bbox: x1:275    y1:-1    x2:361    y2:52
Detected: rider conf: 0.47  bbox: x1:24    y1:-1    x2:162    y2:350
Detected: rider conf: 

In [1]:
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
import cv2
import time
from my_functions import *

notice_generated = False  # Flag to track if notice has been generated

def generate_notice(vehicle_number):
    global notice_generated  # Use global variable
    if not notice_generated:
        if vehicle_number.strip() == "":
            messagebox.showerror("Error", "Please enter the vehicle number.")
        else:
            notice = f"Notice: You were detected riding without a helmet and number plate on {datetime.now()}. Please ensure you wear a helmet for your safety and to comply with traffic regulations. Violation recorded with vehicle number: {vehicle_number}."
            messagebox.showinfo("Notice Generated", notice)
        notice_generated = True  # Set flag to True after generating notice

def process_detection(frame, results):
    global notice_generated  # Use global variable
    rider_list = []
    head_list = []
    number_list = []

    for result in results:
        x1,y1,x2,y2,cnf, clas = result
        if clas == 0:
            rider_list.append(result)
        elif clas == 1:
            head_list.append(result)
        elif clas == 2:
            number_list.append(result)

    for rdr in rider_list:
        x1r, y1r, x2r, y2r, cnfr, clasr = rdr
        for hd in head_list:
            x1h, y1h, x2h, y2h, cnfh, clash = hd
            if inside_box([x1r,y1r,x2r,y2r], [x1h,y1h,x2h,y2h]): 
                try:
                    head_img = frame[y1h:y2h, x1h:x2h]
                    helmet_present = img_classify(head_img)
                except:
                    helmet_present[0] = None

                if helmet_present[0] == False:
                    for num in number_list:
                        x1_num, y1_num, x2_num, y2_num, conf_num, clas_num = num
                        if inside_box([x1r,y1r,x2r,y2r], [x1_num, y1_num, x2_num, y2_num]):
                            vehicle_number = extract_number_plate(frame, num)
                            generate_notice(vehicle_number)
                            return
    # If no violation detected or notice already generated, return None
    return None

def main():
    source = 'video.mp4' 
    save_video = True
    show_video = True
    save_img = False
    frame_size = (640, 480)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi', fourcc, 20.0, frame_size)

    cap = cv2.VideoCapture(source)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frame = cv2.resize(frame, frame_size)
            original_frame = frame.copy()
            frame, results = object_detection(frame)

            if process_detection(original_frame, results):
                # If violation detected, process_detection will generate notice
                pass

            if save_video:
                out.write(frame)
            if save_img:
                cv2.imwrite('saved_frame.jpg', frame)
            if show_video:
                cv2.imshow('Frame', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        else:
            break

    cap.release()
    #cv2.destroyAllWindows()
    print('Execution completed')

if __name__ == "__main__":
    main()


C:\Users\i\anaconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Detected: head conf: 0.43  bbox: x1:436    y1:15    x2:536    y2:140
Detected: rider conf: 0.37  bbox: x1:414    y1:3    x2:576    y2:477
Detected: head conf: 0.62  bbox: x1:442    y1:311    x2:529    y2:414
Detected: rider conf: 0.56  bbox: x1:393    y1:301    x2:582    y2:477
Detected: head conf: 0.38  bbox: x1:397    y1:285    x2:574    y2:467
Detected: head conf: 0.65  bbox: x1:445    y1:200    x2:525    y2:306
Detected: rider conf: 0.59  bbox: x1:403    y1:191    x2:582    y2:478
Detected: rider conf: 0.66  bbox: x1:257    y1:334    x2:459    y2:480
Detected: head conf: 0.43  bbox: x1:307    y1:337    x2:385    y2:428
Detected: rider conf: 0.54  bbox: x1:235    y1:286    x2:460    y2:473
Detected: head conf: 0.4  bbox: x1:281    y1:303    x2:378    y2:389
Detected: rider conf: 0.56  bbox: x1:228    y1:261    x2:432    y2:479
Detected: rider conf: 0.46  bbox: x1:228    y1:233    x2:419    y2:469
Detected: rider conf: 0.37  bbox: x1:223    y1:183    x2:399    y2:475
Detected: rider 

Detected: rider conf: 0.36  bbox: x1:124    y1:263    x2:310    y2:483
Detected: rider conf: 0.63  bbox: x1:-6    y1:331    x2:172    y2:480
Detected: rider conf: 0.54  bbox: x1:5    y1:300    x2:184    y2:484
Detected: rider conf: 0.42  bbox: x1:14    y1:245    x2:204    y2:495
Detected: rider conf: 0.59  bbox: x1:39    y1:229    x2:214    y2:488
Detected: rider conf: 0.66  bbox: x1:51    y1:186    x2:225    y2:482
Detected: rider conf: 0.61  bbox: x1:65    y1:156    x2:235    y2:482
Detected: rider conf: 0.5  bbox: x1:139    y1:328    x2:338    y2:469
Detected: rider conf: 0.42  bbox: x1:-5    y1:210    x2:189    y2:472
Detected: rider conf: 0.68  bbox: x1:176    y1:310    x2:372    y2:476
Detected: rider conf: 0.68  bbox: x1:183    y1:261    x2:373    y2:476
Detected: rider conf: 0.55  bbox: x1:193    y1:216    x2:369    y2:475
Detected: rider conf: 0.46  bbox: x1:49    y1:-4    x2:171    y2:287
Detected: rider conf: 0.51  bbox: x1:76    y1:366    x2:270    y2:476
Detected: rider co

Detected: rider conf: 0.4  bbox: x1:-1    y1:346    x2:217    y2:478
Detected: rider conf: 0.4  bbox: x1:-9    y1:237    x2:196    y2:491
Detected: rider conf: 0.48  bbox: x1:-1    y1:-4    x2:87    y2:205
Detected: rider conf: 0.41  bbox: x1:118    y1:337    x2:310    y2:476
Detected: rider conf: 0.67  bbox: x1:307    y1:176    x2:462    y2:484
Detected: head conf: 0.5  bbox: x1:356    y1:180    x2:419    y2:274
Detected: head conf: 0.51  bbox: x1:84    y1:438    x2:152    y2:482
Detected: rider conf: 0.4  bbox: x1:69    y1:349    x2:200    y2:477
Detected: head conf: 0.37  bbox: x1:74    y1:339    x2:193    y2:481
Detected: rider conf: 0.58  bbox: x1:65    y1:304    x2:225    y2:482
Detected: rider conf: 0.54  bbox: x1:81    y1:257    x2:224    y2:487
Detected: rider conf: 0.4  bbox: x1:88    y1:204    x2:224    y2:485
Detected: rider conf: 0.57  bbox: x1:564    y1:44    x2:638    y2:491
Detected: rider conf: 0.56  bbox: x1:552    y1:-1    x2:637    y2:481
Detected: rider conf: 0.42 

Detected: rider conf: 0.52  bbox: x1:182    y1:358    x2:354    y2:478
Detected: rider conf: 0.42  bbox: x1:163    y1:319    x2:337    y2:474
Detected: rider conf: 0.37  bbox: x1:158    y1:274    x2:327    y2:487
Detected: rider conf: 0.4  bbox: x1:154    y1:256    x2:325    y2:485
Detected: rider conf: 0.38  bbox: x1:158    y1:211    x2:303    y2:484
Detected: rider conf: 0.51  bbox: x1:352    y1:1    x2:471    y2:109
Execution completed


In [1]:
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
import cv2
import time
from my_functions import *

def generate_notice(vehicle_number, image):
    if vehicle_number.strip() == "":
        messagebox.showerror("Error", "Please enter the vehicle number.")
    else:
        notice = f"Notice: You were detected riding without a helmet and number plate on {datetime.now()}. Please ensure you wear a helmet for your safety and to comply with traffic regulations. Violation recorded with vehicle number: {vehicle_number}."
        messagebox.showinfo("Notice Generated", notice)

        # Create a new Tkinter window to display the image
        notice_window = tk.Toplevel()
        notice_window.title("Violation Image")
        image_label = tk.Label(notice_window, image=image)
        image_label.pack()

def process_detection(frame, results):
    rider_list = []
    head_list = []
    number_list = []

    for result in results:
        x1,y1,x2,y2,cnf, clas = result
        if clas == 0:
            rider_list.append(result)
        elif clas == 1:
            head_list.append(result)
        elif clas == 2:
            number_list.append(result)

    for rdr in rider_list:
        x1r, y1r, x2r, y2r, cnfr, clasr = rdr
        for hd in head_list:
            x1h, y1h, x2h, y2h, cnfh, clash = hd
            if inside_box([x1r,y1r,x2r,y2r], [x1h,y1h,x2h,y2h]): 
                try:
                    head_img = frame[y1h:y2h, x1h:x2h]
                    helmet_present = img_classify(head_img)
                except:
                    helmet_present[0] = None

                if helmet_present[0] == False:
                    for num in number_list:
                        x1_num, y1_num, x2_num, y2_num, conf_num, clas_num = num
                        if inside_box([x1r,y1r,x2r,y2r], [x1_num, y1_num, x2_num, y2_num]):
                            vehicle_number = extract_number_plate(frame, num)
                            generate_notice(vehicle_number, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                            return
    # If no violation detected, return None
    return None

def main():
    source = 'video.mp4' 
    save_video = True
    show_video = True
    save_img = False
    frame_size = (640, 480)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi', fourcc, 20.0, frame_size)

    cap = cv2.VideoCapture(source)

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frame = cv2.resize(frame, frame_size)
            original_frame = frame.copy()
            frame, results = object_detection(frame)

            if process_detection(original_frame, results):
                # If violation detected, process_detection will generate notice
                pass

            if save_video:
                out.write(frame)
            if save_img:
                cv2.imwrite('saved_frame.jpg', frame)
            if show_video:
                cv2.imshow('Frame', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        else:
            break

    cap.release()
    cv2.destroyAllWindows()
    print('Execution completed')

if __name__ == "__main__":
    main()


C:\Users\i\anaconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Detected: head conf: 0.43  bbox: x1:436    y1:15    x2:536    y2:140
Detected: rider conf: 0.37  bbox: x1:414    y1:3    x2:576    y2:477
Detected: head conf: 0.62  bbox: x1:442    y1:311    x2:529    y2:414
Detected: rider conf: 0.56  bbox: x1:393    y1:301    x2:582    y2:477
Detected: head conf: 0.38  bbox: x1:397    y1:285    x2:574    y2:467
Detected: head conf: 0.65  bbox: x1:445    y1:200    x2:525    y2:306
Detected: rider conf: 0.59  bbox: x1:403    y1:191    x2:582    y2:478
Detected: rider conf: 0.66  bbox: x1:257    y1:334    x2:459    y2:480
Detected: head conf: 0.43  bbox: x1:307    y1:337    x2:385    y2:428
Detected: rider conf: 0.54  bbox: x1:235    y1:286    x2:460    y2:473
Detected: head conf: 0.4  bbox: x1:281    y1:303    x2:378    y2:389
Detected: rider conf: 0.56  bbox: x1:228    y1:261    x2:432    y2:479
Detected: rider conf: 0.46  bbox: x1:228    y1:233    x2:419    y2:469
Detected: rider conf: 0.37  bbox: x1:223    y1:183    x2:399    y2:475
Detected: rider 

Detected: rider conf: 0.36  bbox: x1:124    y1:263    x2:310    y2:483
Detected: rider conf: 0.63  bbox: x1:-6    y1:331    x2:172    y2:480
Detected: rider conf: 0.54  bbox: x1:5    y1:300    x2:184    y2:484
Detected: rider conf: 0.42  bbox: x1:14    y1:245    x2:204    y2:495
Detected: rider conf: 0.59  bbox: x1:39    y1:229    x2:214    y2:488
Detected: rider conf: 0.66  bbox: x1:51    y1:186    x2:225    y2:482
Detected: rider conf: 0.61  bbox: x1:65    y1:156    x2:235    y2:482
Detected: rider conf: 0.5  bbox: x1:139    y1:328    x2:338    y2:469
Detected: rider conf: 0.42  bbox: x1:-5    y1:210    x2:189    y2:472
Detected: rider conf: 0.68  bbox: x1:176    y1:310    x2:372    y2:476
Detected: rider conf: 0.68  bbox: x1:183    y1:261    x2:373    y2:476
Detected: rider conf: 0.55  bbox: x1:193    y1:216    x2:369    y2:475
Detected: rider conf: 0.46  bbox: x1:49    y1:-4    x2:171    y2:287
Detected: rider conf: 0.51  bbox: x1:76    y1:366    x2:270    y2:476
Detected: rider co

Detected: rider conf: 0.4  bbox: x1:-1    y1:346    x2:217    y2:478
Detected: rider conf: 0.4  bbox: x1:-9    y1:237    x2:196    y2:491
Detected: rider conf: 0.48  bbox: x1:-1    y1:-4    x2:87    y2:205
Detected: rider conf: 0.41  bbox: x1:118    y1:337    x2:310    y2:476
Detected: rider conf: 0.67  bbox: x1:307    y1:176    x2:462    y2:484
Detected: head conf: 0.5  bbox: x1:356    y1:180    x2:419    y2:274
Detected: head conf: 0.51  bbox: x1:84    y1:438    x2:152    y2:482
Detected: rider conf: 0.4  bbox: x1:69    y1:349    x2:200    y2:477
Detected: head conf: 0.37  bbox: x1:74    y1:339    x2:193    y2:481
Detected: rider conf: 0.58  bbox: x1:65    y1:304    x2:225    y2:482
Detected: rider conf: 0.54  bbox: x1:81    y1:257    x2:224    y2:487
Detected: rider conf: 0.4  bbox: x1:88    y1:204    x2:224    y2:485
Detected: rider conf: 0.57  bbox: x1:564    y1:44    x2:638    y2:491
Detected: rider conf: 0.56  bbox: x1:552    y1:-1    x2:637    y2:481
Detected: rider conf: 0.42 

Detected: rider conf: 0.52  bbox: x1:182    y1:358    x2:354    y2:478
Detected: rider conf: 0.42  bbox: x1:163    y1:319    x2:337    y2:474
Detected: rider conf: 0.37  bbox: x1:158    y1:274    x2:327    y2:487
Detected: rider conf: 0.4  bbox: x1:154    y1:256    x2:325    y2:485
Detected: rider conf: 0.38  bbox: x1:158    y1:211    x2:303    y2:484
Detected: rider conf: 0.51  bbox: x1:352    y1:1    x2:471    y2:109
Detected: head conf: 0.64  bbox: x1:58    y1:352    x2:185    y2:475
Detected: rider conf: 0.62  bbox: x1:47    y1:358    x2:198    y2:477
Detected: rider conf: 0.47  bbox: x1:33    y1:307    x2:191    y2:479
Detected: rider conf: 0.56  bbox: x1:31    y1:259    x2:187    y2:480
Detected: head conf: 0.44  bbox: x1:62    y1:262    x2:144    y2:346
Detected: rider conf: 0.47  bbox: x1:21    y1:215    x2:178    y2:478
Detected: rider conf: 0.55  bbox: x1:442    y1:343    x2:630    y2:476
Detected: head conf: 0.53  bbox: x1:461    y1:332    x2:611    y2:478
Detected: rider co